In [1]:
%pwd

'e:\\github_clone\\Text-Summarization-English\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'e:\\github_clone\\Text-Summarization-English'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    save_steps: str
    gradient_accumulation_steps: int
    logging_steps: int
    push_to_hub: bool
    report_to: list

In [5]:
from textSummerizerEnglish.logging import logger
from textSummerizerEnglish.constants import *
from textSummerizerEnglish.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            push_to_hub=params.push_to_hub,
            report_to=params.report_to,
        )
        return model_trainer_config

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer, pipeline
from datasets import load_dataset, Dataset, load_from_disk
import torch

c:\Users\Ernest\.conda\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_flan = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_flan)

        train_dataset = load_from_disk(Path(os.path.join(self.config.data_path, "new_summary", "test")))
        val_dataset = load_from_disk(Path(os.path.join(self.config.data_path, "new_summary", "val")))

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            push_to_hub=self.config.push_to_hub,
            report_to=[]
        )

        trainer = Trainer(model=model_flan, 
                          args=trainer_args,
                          tokenizer=tokenizer, 
                          data_collator=seq2seq_data_collator,
                          train_dataset=train_dataset, 
                          eval_dataset=val_dataset)
        trainer.train()

        model_flan.save_pretrained(Path(os.path.join(self.config.root_dir,"Flan-T5-model")))
        ## Save tokenizer
        tokenizer.save_pretrained(Path(os.path.join(self.config.root_dir,"tokenizer")))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-04 23:49:56,723: INFO: yaml file : config\config.yaml loaded successfully]
[2023-12-04 23:49:56,726: INFO: yaml file : params.yaml loaded successfully]
[2023-12-04 23:49:56,729: INFO: Created directory at artifacts]
[2023-12-04 23:49:56,729: INFO: Created directory at artifacts/model_trainer]


 69%|██████▉   | 200/288 [06:29<02:51,  1.95s/it]

{'loss': 2.1356, 'learning_rate': 2e-05, 'epoch': 4.17}


100%|██████████| 288/288 [09:21<00:00,  1.95s/it]


{'train_runtime': 561.9614, 'train_samples_per_second': 8.2, 'train_steps_per_second': 0.512, 'train_loss': 2.090376165178087, 'epoch': 6.0}
